In [4]:
# General imports
import numpy as np
import pandas as pd
import os, warnings, math

from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings('ignore')

In [5]:
########################### Helpers
#################################################################################
## -------------------
## Memory Reducer
# :df pandas dataframe to reduce size             # type: pd.DataFrame()
# :verbose                                        # type: bool
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df
## -------------------

In [6]:
########################### DATA LOAD
#################################################################################
print('Load Data')
train_df = pd.read_csv('../input/ashrae-energy-prediction/train.csv')
test_df = pd.read_csv('../input/ashrae-energy-prediction/test.csv')

building_df = pd.read_csv('../input/ashrae-energy-prediction/building_metadata.csv')

train_weather_df = pd.read_csv('../input/ashrae-energy-prediction/weather_train.csv')
test_weather_df = pd.read_csv('../input/ashrae-energy-prediction/weather_test.csv')

Load Data


In [7]:
########################### Data Check
#################################################################################
print('Main data:', list(train_df), train_df.info())
print('#'*20)

print('Buildings data:',list(building_df), building_df.info())
print('#'*20)

print('Weather data:',list(train_weather_df), train_weather_df.info())
print('#'*20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 4 columns):
building_id      int64
meter            int64
timestamp        object
meter_reading    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 616.9+ MB
Main data: ['building_id', 'meter', 'timestamp', 'meter_reading'] None
####################
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1449 entries, 0 to 1448
Data columns (total 6 columns):
site_id        1449 non-null int64
building_id    1449 non-null int64
primary_use    1449 non-null object
square_feet    1449 non-null int64
year_built     675 non-null float64
floor_count    355 non-null float64
dtypes: float64(2), int64(3), object(1)
memory usage: 68.0+ KB
Buildings data: ['site_id', 'building_id', 'primary_use', 'square_feet', 'year_built', 'floor_count'] None
####################
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139773 entries, 0 to 139772
Data columns (total 9 columns):
site_id        

In [8]:
########################### Convert timestamp to date
#################################################################################
for df in [train_df, test_df, train_weather_df, test_weather_df]:
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
for df in [train_df, test_df]:
    # 年中第几月
    df['DT_M'] = df['timestamp'].dt.month.astype(np.int8)
    # 年中第几周
    df['DT_W'] = df['timestamp'].dt.weekofyear.astype(np.int8)
    # 年中第几天
    df['DT_D'] = df['timestamp'].dt.dayofyear.astype(np.int16)
    
    # 小时，0点至23点
    df['DT_hour'] = df['timestamp'].dt.hour.astype(np.int8)
    # 周几，0是周一，5是周六，6是周天
    df['DT_day_week'] = df['timestamp'].dt.dayofweek.astype(np.int8)
    # 每个月的几号
    df['DT_day_month'] = df['timestamp'].dt.day.astype(np.int8)
    # 一个月中第几周, 感觉有些多余，能知道第五周之后是新月份的第一周吗？
    df['DT_week_month'] = df['timestamp'].dt.day/7
    df['DT_week_month'] = df['DT_week_month'].apply(lambda x: math.ceil(x)).astype(np.int8)

In [9]:
########################### Strings to category
#################################################################################
building_df['primary_use'] = building_df['primary_use'].astype('category')

In [10]:
########################### Building Transform
#################################################################################
# 这里的空值先不处理
# building_df['floor_count'] = building_df['floor_count'].fillna(0).astype(np.int8)
# building_df['year_built'] = building_df['year_built'].fillna(-999).astype(np.int16)

le = LabelEncoder()
building_df['primary_use'] = building_df['primary_use'].astype(str)
building_df['primary_use'] = le.fit_transform(building_df['primary_use']).astype(np.int8)

In [11]:
train_df.meter_reading

0             0.000
1             0.000
2             0.000
3             0.000
4             0.000
             ...   
20216095      8.750
20216096      4.825
20216097      0.000
20216098    159.575
20216099      2.850
Name: meter_reading, Length: 20216100, dtype: float64

In [12]:
train_weather_df.air_temperature

0         25.0
1         24.4
2         22.8
3         21.1
4         20.0
          ... 
139768     3.0
139769     2.8
139770     2.8
139771     2.2
139772     1.7
Name: air_temperature, Length: 139773, dtype: float64

In [13]:
train_weather_df.dew_temperature

0         20.0
1         21.1
2         21.1
3         20.6
4         20.0
          ... 
139768    -8.0
139769    -8.9
139770    -7.2
139771    -6.7
139772    -5.6
Name: dew_temperature, Length: 139773, dtype: float64

In [14]:
train_weather_df.sea_level_pressure

0         1019.7
1         1020.2
2         1020.2
3         1020.1
4         1020.0
           ...  
139768       NaN
139769    1007.4
139770    1007.5
139771    1008.0
139772    1008.5
Name: sea_level_pressure, Length: 139773, dtype: float64

In [15]:
train_weather_df.wind_direction

0           0.0
1          70.0
2           0.0
3           0.0
4         250.0
          ...  
139768    180.0
139769    180.0
139770    180.0
139771    170.0
139772    180.0
Name: wind_direction, Length: 139773, dtype: float64

In [16]:
########################### Base check
#################################################################################
do_not_convert = ['category','datetime64[ns]','object']
for df in [train_df, test_df, building_df, train_weather_df, test_weather_df]:
    original = df.copy()
    df = reduce_mem_usage(df)

    for col in list(df):
        if df[col].dtype.name not in do_not_convert:
            if (df[col]-original[col]).sum()!=0:
                df[col] = original[col]
                print('Bad transformation', col)

Mem. usage decreased to 443.43 Mb (42.5% reduction)
Bad transformation meter_reading
Mem. usage decreased to 914.62 Mb (42.5% reduction)
Mem. usage decreased to  0.02 Mb (70.6% reduction)
Mem. usage decreased to  3.07 Mb (68.1% reduction)
Bad transformation air_temperature
Bad transformation dew_temperature
Bad transformation sea_level_pressure
Bad transformation wind_speed
Mem. usage decreased to  6.08 Mb (68.1% reduction)
Bad transformation air_temperature
Bad transformation dew_temperature
Bad transformation sea_level_pressure
Bad transformation wind_speed


In [17]:
train_df.meter_reading

0             0.000
1             0.000
2             0.000
3             0.000
4             0.000
             ...   
20216095      8.750
20216096      4.825
20216097      0.000
20216098    159.575
20216099      2.850
Name: meter_reading, Length: 20216100, dtype: float64

In [18]:
train_weather_df.air_temperature

0         25.0
1         24.4
2         22.8
3         21.1
4         20.0
          ... 
139768     3.0
139769     2.8
139770     2.8
139771     2.2
139772     1.7
Name: air_temperature, Length: 139773, dtype: float64

In [19]:
train_weather_df.dew_temperature

0         20.0
1         21.1
2         21.1
3         20.6
4         20.0
          ... 
139768    -8.0
139769    -8.9
139770    -7.2
139771    -6.7
139772    -5.6
Name: dew_temperature, Length: 139773, dtype: float64

In [20]:
train_weather_df.sea_level_pressure

0         1019.7
1         1020.2
2         1020.2
3         1020.1
4         1020.0
           ...  
139768       NaN
139769    1007.4
139770    1007.5
139771    1008.0
139772    1008.5
Name: sea_level_pressure, Length: 139773, dtype: float64

In [21]:
train_weather_df.wind_direction

0           0.0
1          70.0
2           0.0
3           0.0
4         250.0
          ...  
139768    180.0
139769    180.0
139770    180.0
139771    170.0
139772    180.0
Name: wind_direction, Length: 139773, dtype: float16

In [22]:
########################### Data Check
#################################################################################
print('Main data:', list(train_df), train_df.info())
print('#'*20)

print('Buildings data:',list(building_df), building_df.info())
print('#'*20)

print('Weather data:',list(train_weather_df), train_weather_df.info())
print('#'*20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 11 columns):
building_id      int16
meter            int8
timestamp        datetime64[ns]
meter_reading    float64
DT_M             int8
DT_W             int8
DT_D             int16
DT_hour          int8
DT_day_week      int8
DT_day_month     int8
DT_week_month    int8
dtypes: datetime64[ns](1), float64(1), int16(2), int8(7)
memory usage: 520.5 MB
Main data: ['building_id', 'meter', 'timestamp', 'meter_reading', 'DT_M', 'DT_W', 'DT_D', 'DT_hour', 'DT_day_week', 'DT_day_month', 'DT_week_month'] None
####################
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1449 entries, 0 to 1448
Data columns (total 6 columns):
site_id        1449 non-null int8
building_id    1449 non-null int16
primary_use    1449 non-null int8
square_feet    1449 non-null int32
year_built     675 non-null float16
floor_count    355 non-null float16
dtypes: float16(2), int16(1), int32(1), int8(2)
memory us

In [23]:
########################### Export (using same names as in competition dataset)
#################################################################################
train_df.to_pickle('../input/as-data-minification/train.pkl')
test_df.to_pickle('../input/as-data-minification/test.pkl')

building_df.to_pickle('../input/as-data-minification/building_metadata.pkl')

train_weather_df.to_pickle('../input/as-data-minification/weather_train.pkl')
test_weather_df.to_pickle('../input/as-data-minification/weather_test.pkl')